# PELIC spelling

This notebook adds further processing to `PELIC_compiled.csv`  in the [`PELIC-dataset`](https://github.com/ELI-Data-Mining-Group/PELIC-dataset) repo by creating a column of tok_POS whose spelling has been automatically corrected.

**Notebook contents:**
- [Building `non_words_df`](#Building-non_words_df)
- [Building `misspell_df`](#Building-misspell_df)
- [Possible segmentation](#Applying-segmentation)
- [Applying spelling correction](#Applying-spelling-correction)
- [Incorporating corrections into `pelic_df`](#Incorporating-corrections-into-pelic_df)

## Building non_words_df
In this section, we build a dataframe, `non_words_df`, which collects all of the non-words from the PELIC dataset (in `PELIC_compiled.csv`). The final dataframe has the following columns:
- `non_word`: tuples with the non-words and their parts of speech
- `sentence`: the complete sentence containing the non-word to provide context
- `answer_id`: the id of the text they come from

In [1]:
# Import necessary modules

import pandas as pd
import pprint
import numpy as np
from ast import literal_eval
import nltk
import random
from pelitk import lex

In [2]:
# Read in PELIC_compiled.csv

pelic_df = pd.read_csv("../PELIC-dataset/PELIC_compiled.csv", index_col = 'answer_id', # answer_id is unique
                      dtype = {'level_id':'object','question_id':'object','version':'object','course_id':'object'}, # str not ints
                               converters={'tokens':literal_eval,'tok_lem_POS':literal_eval}) # read in as lists
pelic_df.head()

,anon_id,L1,gender,course_id,level_id,class_id,question_id,version,text_len,text,tokens,tok_lem_POS
answer_id,,,,,,,,,,,,
1,eq0,Arabic,Male,149,4,g,5,1,177,I met my friend Nife while I was studying in a...,"[I, met, my, friend, Nife, while, I, was, stud...","((I, i, PRP), (met, meet, VBD), (my, my, PRP$)..."
2,am8,Thai,Female,149,4,g,5,1,137,"Ten years ago, I met a women on the train betw...","[Ten, years, ago, ,, I, met, a, women, on, the...","((Ten, ten, CD), (years, year, NNS), (ago, ago..."
3,dk5,Turkish,Female,115,4,w,12,1,64,In my country we usually don't use tea bags. F...,"[In, my, country, we, usually, do, n't, use, t...","((In, in, IN), (my, my, PRP$), (country, count..."
4,dk5,Turkish,Female,115,4,w,13,1,6,I organized the instructions by time.,"[I, organized, the, instructions, by, time, .]","((I, i, PRP), (organized, organize, VBD), (the..."
5,ad1,Korean,Female,115,4,w,12,1,59,"First, prepare a port, loose tea, and cup.\nSe...","[First, ,, prepare, a, port, ,, loose, tea, ,,...","((First, first, RB), (,, ,, ,), (prepare, prep..."


The focus here is the `tok_lem_POS` column, but all columns will be kept as the entire df will be written out at the end of the notebook.

In [3]:
# Creating small dataframe to be used for finding non-words

non_words = pelic_df[['text','tok_lem_POS']]

**Note:** For spelling correction, it is necessary to decide what list of words will be used for determining if a word is real or not.

Here, we use the [`SCOWL_condensed.txt`](https://github.com/ELI-Data-Mining-Group/PELIC-spelling/blob/master/SCOWL_condensed.txt) file which is a combination of wordlists available for download at http://wordlist.aspell.net/. We include items from all the dictionaries _except_ the abbreviations dictionary. For a detailed look at the compilation of this dictionary, please see the [SCOWL_wordlist](https://github.com/ELI-Data-Mining-Group/PELIC-spelling/blob/master/SCOWL_wordlist.ipynb) notebook.

In [4]:
#Reading in SCOWL_condensed as a set as a lookup list for spelling (500k words)

scowl = set(open("SCOWL_condensed.txt", "r").read().split('\n'))
print(random.sample(scowl,5))

['resublime', 'nonabidingly', 'idocrases', 'Lebar', 'lotic']


In [5]:
scowl = set([x.lower() for x in scowl])
len(scowl)

497552

The following is a list of words which should be considered words but which were previously being labelled as non-words. These items have been manually added to this list based on output later in this notebook. Most of these items are food items, names, or abbreviations.

In [6]:
scowl_supp = open("SCOWL_supp.txt", "r").read().split(',')
scowl_supp = [x[2:-1] for x in scowl_supp]
print(len(scowl_supp))
print(scowl_supp)

233
['adha', 'adj', 'ahamed', 'alaikum', 'anonurlpage', 'antiretroviral', 'arpa', 'beyonce', 'bibimbap', 'bio', 'biodiesel', 'bioethanol', 'bulgogi', 'bundang', 'cafe', 'carnaval', 'cds', 'cf', 'co', 'comscore', 'cyber', 'ddukboggi', 'def', 'dr', 'eg', 'eid', 'electrospray', 'entrees', 'erectus', 'etc', 'fiance', 'fiancee', 'fiter', 'fitir', 'fitr', 'fl', 'freediving', 'fukubukuro', 'geolinguist', 'hikikomori', 'hp', 'ibt', 'iq', 'iriver', 'jetta', 'jul', 'kabsa', 'kaled', 'kawader', 'km', 'leisureville', 'll', 'maamool', 'mayumi', 'mcdonalds', 'min', 'mongongo', 'nc', 'neuro', 'nian', 'notting', 'okroshka', 'onsen', 'pajeon', 'pbt', 'pc', 'pcs', 'pp', 'pudim', 'puket', 'samear', 'shui', 'sq', 'st', 'staycation', 'sth', 'taoyuan', 'toefl', 'trans', 'transgene', 'tv', 'unsub', 'va', 'vol', 'vs', 'webaholic', 'webaholics', 'webaholism', 'wenjing', 'woong', 'yaoming', 'ying', 'yingdong', 'yugong', 'yuval', 'zi', 'abha', 'achuar', 'ae', 'afandi', 'aladha', 'alfater', 'alfeter', 'alfetr', '

In [7]:
# Lower case all toks

non_words.tok_lem_POS = non_words.tok_lem_POS.apply(lambda row: [(x[0].lower(),x[1],x[2]) for x in row])

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [8]:
# Function to find non-words

def spell_check(tok_lem_POS_list):
    word_list = scowl # Choose word_list here. Default is scowl described above.
    not_in_word_list = []
    for tok_lem_POS in tok_lem_POS_list:
        if tok_lem_POS[0] not in word_list and tok_lem_POS[0] not in scowl_supp:
            not_in_word_list.append(tok_lem_POS)
    return not_in_word_list

In [9]:
# Apply spell check function to find all misspelled-words. 

non_words['misspelled_words'] = non_words.tok_lem_POS.apply(spell_check)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
non_words.head()

,text,tok_lem_POS,misspelled_words
answer_id,,,
1,I met my friend Nife while I was studying in a...,"[(i, i, PRP), (met, meet, VBD), (my, my, PRP$)...","[(., ., .), (., ., .), (., ., .), (;, ;, :), (..."
2,"Ten years ago, I met a women on the train betw...","[(ten, ten, CD), (years, year, NNS), (ago, ago...","[(,, ,, ,), (,, ,, ,), (., ., .), (;, ;, :), (..."
3,In my country we usually don't use tea bags. F...,"[(in, in, IN), (my, my, PRP$), (country, count...","[(., ., .), (,, ,, ,), (., ., .), (., ., .), (..."
4,I organized the instructions by time.,"[(i, i, PRP), (organized, organize, VBD), (the...","[(., ., .)]"
5,"First, prepare a port, loose tea, and cup.\nSe...","[(first, first, RB), (,, ,, ,), (prepare, prep...","[(,, ,, ,), (,, ,, ,), (,, ,, ,), (., ., .), (..."


#### Adding context to the dataframe
Seeing the mistakes in the context of a sentence will allow for better manual checking if required.

In [11]:
# Sent-tokenizing the text

non_words['sentence'] = non_words['text'].apply(lambda x: nltk.sent_tokenize(x))

# And delete text column which is no longer needed

del non_words['text']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
# Checking for hyphenated words tagged as misspellings because SCOWL doesn't contain hypenated words

hyphenated = set([x[0] for x in [x for y in non_words.misspelled_words.to_list() for x in y] if '-' in x[0]])
print(len(hyphenated))
print(list(hyphenated)[:10])

# These need to be removed from the non-words dataframe if composed of valid words

1182
['half-time', 'upper-middle', 'lower-back', 'light-brown', 'one-parent', 'operating-room', 'post-impressionist', 'intra-abdominal', 'role-play', 'cost-effective']


In [13]:
# Hyphenated items whose components are not in scowl - possible misspellings or punctuation strings

sorted([y for y in [x.split('-') for x in hyphenated] if y[0] not in scowl or y[1] not in scowl])

[['', "'"],
 ['', '***', '****'],
 ['', '+'],
 ['', '.'],
 ['',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  ''],
 ["'", ''],
 ['.', ''],
 ['/', ''],
 ['\\\\', ''],
 ['^', '^'],
 ['al', 'qaida'],
 ['austro', 'hungarian'],
 ['cd', 'rom'],
 ['co', 'authored'],
 ['co', 'ed'],
 ['co', 'educational'],
 ['co', 'exist'],
 ['co', 'existence'],
 ['co', 'founded'],
 ['co', 'founder'],
 ['co', 'founders'],
 ['co', 'host'],
 ['co', 'op'],
 ['co', 'operate'],
 ['co', 'operation'],
 ['co', 'pay'],
 ['co', 'pilot'],
 ['co', 'sleeping'],
 ['co', 'star'],
 ['co', 'worker'],
 ['co', 'workers'],
 ['co', 'written'],
 ['co', 'wrote'],
 ['mah', 'jong'],
 ['mid', '80s'],
 ['pay', 'tv'],
 ['roly', 'poly'],
 ['socio', 'cultural'],
 ['socio', 'economic'],
 ['trans', 'fat'],
 ['vis', 'a', 'vis'],
 ['wal', 'mart']]

After manual checking, all the hypenated words are punctuation, real words (or true productive use of affixes) and can be removed from the non-words df.

The following two cells 
1. remove all the hypenated words from the dataframe
2. remove all words that don't contain a letter

However, as all hyphenated word are fine, we will instead just eliminate all words that are not purely composed of letters. This will have the effect of removing the following categories from the dataframe:
- punctuation
- hyphenated words (e.g. well-known)
- contractions (e.g. 'll, 've)
- years (e.g. 1950s)
- ordinals (e.g. 1st, 2nd)

In [14]:
# Removing hypenated words

# non_words.misspelled_words = non_words.misspelled_words.apply(lambda row: [x for x in row if x[0] not in hyphenated])

In [15]:
# Removing items that are only numbers or punctuation
# .isalpha() cannot be used without 'any' as this also removes hyphenated words

# non_words.misspelled_words = non_words.misspelled_words.apply(lambda row: [x for x in row if any(y.isalpha() for y in x[0])])

In [16]:
# Checking initial length of non_words list

print(len([x for y in non_words.misspelled_words.to_list() for x in y]))
print(len(set([x for y in non_words.misspelled_words.to_list() for x in y])))

680998
20394


In [17]:
# Removing items that are not purely alpha

non_words.misspelled_words = non_words.misspelled_words.apply(lambda row: [x for x in row if x[0].isalpha()])

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [18]:
# Checking affect of removal

print(len([x for y in non_words.misspelled_words.to_list() for x in y]))
print(len(set([x for y in non_words.misspelled_words.to_list() for x in y])))

26750
15788


In [19]:
# Removing proper names - NNP, NNPS

# non_words.misspelled_words = non_words.misspelled_words.apply(lambda row: [x for x in row if x[2] != 'NNP' and x[1] != 'NNPS'])

After manual checking, it was decided to keep in items tagged as NNP and NNPS as some items were in fact mistagged and were general capitalized nouns (NN) which were misspelled.

In [20]:
# Checking affect of removal

print(len([x for y in non_words.misspelled_words.to_list() for x in y]))
print(len(set([x for y in non_words.misspelled_words.to_list() for x in y])))

26750
15788


In [21]:
# Removing all words with length 1

non_words.misspelled_words = non_words.misspelled_words.apply(lambda row: [x for x in row if len(x[0]) > 1])

In [22]:
# Checking affect of removal

print(len([x for y in non_words.misspelled_words.to_list() for x in y]))
print(len(set([x for y in non_words.misspelled_words.to_list() for x in y])))

26738
15779


In [23]:
non_words.head()

,tok_lem_POS,misspelled_words,sentence
answer_id,,,
1,"[(i, i, PRP), (met, meet, VBD), (my, my, PRP$)...",[],[I met my friend Nife while I was studying in ...
2,"[(ten, ten, CD), (years, year, NNS), (ago, ago...",[],"[Ten years ago, I met a women on the train bet..."
3,"[(in, in, IN), (my, my, PRP$), (country, count...",[],"[In my country we usually don't use tea bags.,..."
4,"[(i, i, PRP), (organized, organize, VBD), (the...",[],[I organized the instructions by time.]
5,"[(first, first, RB), (,, ,, ,), (prepare, prep...",[],"[First, prepare a port, loose tea, and cup., S..."


Create new dataframe so that each misspelling token is a separate row.

In [449]:
# Removing rows with no misspellings

non_words2 = non_words.loc[non_words.misspelled_words.str.len() > 0,:]

In [450]:
# Exploding the lists in misspelled words so that each misspelling gets its own row

non_words2 = non_words2.explode('misspelled_words')

In [451]:
# Keeping the answer_id (which is no longer unique) as a separate column

non_words2 = non_words2.reset_index(drop = False)

#### Adding a bigrams column, i.e. one token left and right of the misspelled word

In [452]:
# Creating a tokenized version of the sentence without punctuation and with the index for each token

non_words2['enumerated'] = non_words2.sentence.apply(str).apply(lex.re_tokenize).apply(lambda x: list(enumerate(x)))

In [528]:
# Creating a function to extract the bigrams (1 word either side of misspelling)

def get_bigrams(misspelled_word, enumerated_list):
    if len(enumerated_list) <2:
        return []
    for tup in enumerated_list:
        if tup[1] == misspelled_word[0]:
            if tup[0] == 0:
                bigram = ' '.join([x[1] for x in (enumerated_list[tup[0]],enumerated_list[tup[0]+1])])
                return [bigram]
            if tup[0] == len(enumerated_list)-1:
                bigram = ' '.join([x[1] for x in (enumerated_list[tup[0]-1],enumerated_list[tup[0]])])
                return [bigram]
            else:
                bigram1 = ' '.join([x[1] for x in (enumerated_list[tup[0]-1],enumerated_list[tup[0]])])
                bigram2 = ' '.join([x[1] for x in (enumerated_list[tup[0]],enumerated_list[tup[0]+1])])
                return [bigram1, bigram2]

In [529]:
# Testing the function

test_list = non_words2.iloc[5,4]
print(test_list)

first_item = non_words2.iloc[5,1][0] # first item in list
middle_item = non_words2.iloc[5,1][4] # item in in middle of list
last_item = non_words2.iloc[5,1][29] # item at end of list
print('\n',first_item, middle_item, last_item)

[(0, 'every'), (1, 'paragragh'), (2, 's'), (3, 'instructions'), (4, 'depend'), (5, 'on'), (6, 'a'), (7, 'main'), (8, 'idea'), (9, 'but'), (10, 'i'), (11, 'often'), (12, 'organize'), (13, 'the'), (14, 'instruction'), (15, 'by'), (16, 'time'), (17, 'because'), (18, 'i'), (19, 'think'), (20, 'organized'), (21, 'by'), (22, 'time'), (23, 'is'), (24, 'good'), (25, 'looking'), (26, 'for'), (27, 'readers')]

 ('every', 'every', 'DT') ('depend', 'depend', 'VBP') ('readers', 'reader', 'NNS')


In [530]:
print(get_bigrams(first_item,test_list))
print(get_bigrams(middle_item,test_list))
print(get_bigrams(last_item,test_list))

['every paragragh']
['instructions depend', 'depend on']
['for readers']


In [533]:
# Applying the above function
non_words2['bigrams'] = non_words2[['misspelled_words','enumerated']].apply(lambda x: get_bigrams(x[0],x[1]), axis=1)

In [546]:
non_words2.loc[non_words2['bigrams'].isnull()].iloc[1,4]

[(0, 'reading'),
 (1, 'word'),
 (2, 'magazine'),
 (3, 'n'),
 (4, 'njaketribune'),
 (5, 'is'),
 (6, 'the'),
 (7, 'most'),
 (8, 'popular'),
 (9, 'magazine'),
 (10, 'in'),
 (11, 'pittsburgh'),
 (12, 'reading'),
 (13, 'complex'),
 (14, 'n'),
 (15, 'nword'),
 (16, 'complex'),
 (17, 'n'),
 (18, 'nwhat'),
 (19, 'a'),
 (20, 'compplex'),
 (21, 'system'),
 (22, 'reading'),
 (23, 'scene'),
 (24, 'n'),
 (25, 'nthe'),
 (26, 'mountain'),
 (27, 'has'),
 (28, 'z'),
 (29, 'beautiful'),
 (30, 'scene')]

In [544]:
non_words2.loc[non_words2['bigrams'].isnull()]

,answer_id,tok_lem_POS,misspelled_words,sentence,enumerated,bigrams
29,29,"[(to, to, TO), (fail, fail, VB), (a, a, DT), (...","(failling, failling, NN)",[To Fail a Test\n\nFailling on a test is simpl...,"[(0, to), (1, fail), (2, a), (3, test), (4, n)...",None
433,498,"[(reading, read, VBG), (1, 1, CD), (., ., .), ...","(jaketribune, jaketribune, NNP)","[Reading 1., Word : magazine\n\nJakeTribune is...","[(0, reading), (1, word), (2, magazine), (3, n...",None
470,596,"[(commercial, commercial, JJ), (:, :, :), (tod...","(usualy, usualy, VB)",[commercial:\nToday there are many methods for...,"[(0, commercial), (1, ntoday), (2, there), (3,...",None
644,856,"[(the, the, DT), (first, first, JJ), (short, s...","(poète, poète, NNP)",[The first short movie of the American directo...,"[(0, the), (1, first), (2, short), (3, movie),...",None
731,991,"[(comparing, compare, VBG), (guardians, guardi...","(considera, considera, NNP)",[Comparing Guardians\n\nGuardians ANON_NAME_0 ...,"[(0, comparing), (1, guardians), (2, n), (3, n...",None
...,...,...,...,...,...,...
26017,47416,"[(lithium, lithium, NN), ((, (, (), (n, n, JJ)...","(electolyte, electolyte, VBP)","[lithium(n, uncountable) a soft silver-white m...","[(0, lithium), (1, n), (2, uncountable), (3, a...",None
26400,48020,"[(dear, dear, IN), (mr., mr., NNP), (anon_name...","(monira, monira, NNP)",[Dear Mr. ANON_NAME_0\nAs I am a pulmonary reh...,"[(0, dear), (1, mr), (2, anon), (3, name), (4,...",None
26574,48260,"[(dear, dear, NNP), (xavi, xavi, NNP), (,, ,, ...","(fadel, fadel, NNP)","[Dear Xavi,\n\nI am very delighted to hear fro...","[(0, dear), (1, xavi), (2, n), (3, ni), (4, am...",None
26604,48285,"[(dear, dear, IN), (mr., mr., NNP), (xavi, xav...","(abdulkarim, abdulkarim, NNP)",[Dear Mr. xavi\n\nThank you for your email and...,"[(0, dear), (1, mr), (2, xavi), (3, n), (4, nt...",None


In [535]:
# Checking final non_words2 dataframe

non_words2.head()

,answer_id,tok_lem_POS,misspelled_words,sentence,enumerated,bigrams
0,8,"[(i, i, PRP), (organized, organize, VBD), (the...","(beacause, beacause, NN)","[I organized the instructions by time, beacaus...","[(0, i), (1, organized), (2, the), (3, instruc...","[time beacause, beacause to]"
1,11,"[(to, to, TO), (make, make, VB), (tea, tea, NN...","(wallmart, wallmart, NN)","[To make tea, nothing is easier, even if somet...","[(0, to), (1, make), (2, tea), (3, nothing), (...","[in wallmart, wallmart or]"
2,13,"[(first, first, RB), (,, ,, ,), (you, you, PRP...","(dovn, dovn, NN)","[First, you should take some hot water, you ca...","[(0, first), (1, you), (2, should), (3, take),...","[use dovn, dovn mircowave]"
3,13,"[(first, first, RB), (,, ,, ,), (you, you, PRP...","(mircowave, mircowave, VBP)","[First, you should take some hot water, you ca...","[(0, first), (1, you), (2, should), (3, take),...","[dovn mircowave, mircowave or]"
4,15,"[(in, in, IN), (my, my, PRP$), (country, count...","(fitst, fitst, NNP)","[In my country, make a tea is very easy becaus...","[(0, in), (1, my), (2, country), (3, make), (4...","[bags fitst, fitst boil]"


In [28]:
# Total number of non-words (tokens)
print(len(non_words2))

# Total number of non-words (types)
print(non_words2.misspelled_words.nunique())

26738
15779


#### Creating a dataframe of misspellings
In the `non-words2` dataframe above, each row is an occurrence of a misspelling (i.e. _tokens_ ). We also want a dataframe where each row is a misspelling _type_ with frequency information attached.

In [542]:
# Gathering the total misspellings and bigrams

total_misspellings = [x for x in non_words2['misspelled_words']]
total_bigrams = [x for y in non_words2['bigrams'] for x in y] #flattened list

TypeError: 'NoneType' object is not iterable

In [541]:
total_bigrams[:5]

[['time beacause', 'beacause to'],
 ['in wallmart', 'wallmart or'],
 ['use dovn', 'dovn mircowave'],
 ['dovn mircowave', 'mircowave or'],
 ['bags fitst', 'fitst boil']]

In [30]:
# Creating frequency dictionaries for unigrams and bigrams

misspell_freq_dict = {}
for word in total_misspellings:
    if word not in misspell_freq_dict:
        misspell_freq_dict[word] = 1
    else:
        misspell_freq_dict[word] += 1

In [ ]:
misspell_bigram_freq_dict = {}
for bigram in total_misspellings:
    if word not in misspell_freq_dict:
        misspell_freq_dict[word] = 1
    else:
        misspell_freq_dict[word] += 1

In [31]:
print(random.sample(list(misspell_freq_dict),5))

[('tinniger', 'tinniger', 'NN'), ('jonhn', 'jonhn', 'NNP'), ('nitice', 'nitice', 'VB'), ('metala', 'metala', 'NN'), ('tohe', 'tohe', 'JJ')]


In [32]:
# Remove duplicates

final_misspellings = sorted(list(set(total_misspellings)))
len(final_misspellings)

15779

In [33]:
# Constructing misspell_df

misspell_df = pd.DataFrame(final_misspellings)
misspell_df.head()

,0,1,2
0,aa,aa,NNP
1,aa,aa,VB
2,aabout,aabout,IN
3,aad,aad,JJ
4,aain,aain,VBP


In [34]:
# Renaming columns to match other DataFrames in this notebook

misspell_df.rename(columns = {0: 'misspelling',1:'lemma',2:'POS'}, inplace = True)

In [35]:
# Recreating tok_lem_POS column to match dictionary

misspell_df['tok_lem_POS'] = list(zip(misspell_df.misspelling, misspell_df.lemma, misspell_df.POS))
misspell_df.head()

,misspelling,lemma,POS,tok_lem_POS
0,aa,aa,NNP,"(aa, aa, NNP)"
1,aa,aa,VB,"(aa, aa, VB)"
2,aabout,aabout,IN,"(aabout, aabout, IN)"
3,aad,aad,JJ,"(aad, aad, JJ)"
4,aain,aain,VBP,"(aain, aain, VBP)"


In [36]:
# Mapping dictionary to DataFrame

misspell_df['freq'] = misspell_df['tok_lem_POS'].map(misspell_freq_dict)

In [37]:
# Sorting by frequency

misspell_df = misspell_df.sort_values(by=['freq'], ascending=False)

In [38]:
# Resetting index
misspell_df = misspell_df.reset_index(drop = True)
misspell_df.head()

,misspelling,lemma,POS,tok_lem_POS,freq
0,alot,alot,NN,"(alot, alot, NN)",127
1,studing,studing,VBG,"(studing, studing, VBG)",74
2,tofel,tofel,NNP,"(tofel, tofel, NNP)",66
3,eilperin,eilperin,NNP,"(eilperin, eilperin, NNP)",48
4,seollal,seollal,NNP,"(seollal, seollal, NNP)",47


#### scowl_supp
The following is the basis for the 'scowl_supp' list used earlier. Here, errors with a frequency of 10 or more were manually checked, and if determined to be a real word, were added to the scowl_supp list. There were originally 267 items which met this criteria.

In [39]:
print(len(misspell_df.loc[misspell_df.freq >= 10]))
misspell_df.loc[misspell_df.freq >= 10]

144


,misspelling,lemma,POS,tok_lem_POS,freq
0,alot,alot,NN,"(alot, alot, NN)",127
1,studing,studing,VBG,"(studing, studing, VBG)",74
2,tofel,tofel,NNP,"(tofel, tofel, NNP)",66
3,eilperin,eilperin,NNP,"(eilperin, eilperin, NNP)",48
4,seollal,seollal,NNP,"(seollal, seollal, NNP)",47
...,...,...,...,...,...
139,befor,befor,NNP,"(befor, befor, NNP)",10
140,arbus,arbus,NNP,"(arbus, arbus, NNP)",10
141,favorit,favorit,JJ,"(favorit, favorit, JJ)",10
142,peaple,peaple,NN,"(peaple, peaple, NN)",10


### Possible segmentation

Selected segmenter and spellchecker: SymSpell https://github.com/mammothb/symspellpy

There is a dictionary file which which needs to be installed (saved to repo):
[frequency_dictionary_en_82_765.txt](https://symspellpy.readthedocs.io/en/latest/users/installing.html)

To install symspellpy the first time, use pip in command line: `pip install -U symspellpy`

Prior to spelling correct, we first consider using the segmenter. This is a potentially useful first step as misspellings like 'alot' or 'dogmeat' will be separated into 'a lot' and 'dog meat' rather than corrected to a single word like 'lot'.  

However, when segementing misspellings, the segmenter over performs, segmenting non-words into real words where it was clearly not intended, e.g. _improtant_ into _imp rot ant_ or _befor_ into _be for_. As such, the segmenting will not be automated. 

Instead, one manual segmentation will be carried out: _alot_ -> _a lot_ since _alot_ is the most common misspelling remaining in our dataframe (127 occurrences).

In [40]:
# Setting up symspell

from itertools import islice
import pkg_resources
from symspellpy import SymSpell
from symspellpy import Verbosity
sym_spell = SymSpell()
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, 0, 1)

# Print out first 5 elements to demonstrate that dictionary is successfully loaded
list(islice(sym_spell.words.items(), 5))

[('the', 23135851162),
 ('of', 13151942776),
 ('and', 12997637966),
 ('to', 12136980858),
 ('a', 9081174698)]

In [41]:
# Testing segmenter with 'alot' and 'dogmeat'

# Set max_dictionary_edit_distance to avoid spelling correction
sym_spell = SymSpell(max_dictionary_edit_distance=0, prefix_length=7)
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# It is also possible to display frequency with result.distance_sum and edit distance with .log_prob_sum

True

In [42]:
# Creating function for applying the above code

def get_segments(word):
    segments = sym_spell.word_segmentation(word)
    if len(segments.corrected_string.split(' ')) > 1 \
    and segments.corrected_string.split(' ')[0] in scowl and segments.corrected_string.split(' ')[1] in scowl:
        return segments.corrected_string
    else:
        return word

In [43]:
# Testing function

print(get_segments('dogmeat')) # Should be segmented
print(get_segments('fireplace')) # Should not be segmented
print(get_segments('becuase')) # Should not be segmented

dog meat
fireplace
becuase


In [44]:
# Applying the function to create a new column

misspell_df['segments'] =  misspell_df['misspelling'].apply(get_segments)
misspell_df.head(10)

,misspelling,lemma,POS,tok_lem_POS,freq,segments
0,alot,alot,NN,"(alot, alot, NN)",127,a lot
1,studing,studing,VBG,"(studing, studing, VBG)",74,stu ding
2,tofel,tofel,NNP,"(tofel, tofel, NNP)",66,tofel
3,eilperin,eilperin,NNP,"(eilperin, eilperin, NNP)",48,eilperin
4,seollal,seollal,NNP,"(seollal, seollal, NNP)",47,se olla l
5,goverment,goverment,NN,"(goverment, goverment, NN)",47,g over men t
6,decasia,decasia,NNP,"(decasia, decasia, NNP)",45,dec asia
7,iam,iam,NNP,"(iam, iam, NNP)",43,i am
8,misurata,misurata,NNP,"(misurata, misurata, NNP)",37,mi surat a
9,esi,esi,NNP,"(esi, esi, NNP)",34,esi


In [45]:
# Deleting this new column as segmentation creates false segments of misspelled words

del misspell_df['segments']

### Applying spelling correction

In some ways SymSpell is not ideal as sentence context is not considered, only general frequencies. However, other well-known spellcheckers (hunspell, pyspell, etc.) use the same strategy - frequency based criteria for suggestions, without considering immediate cotext. As such, we have followed this common practice, but it is important to remember that accuracy of corrected tokens will not be 100% and must be taken into consideration.

In [106]:
# Testing spelling suggestions with 'becuase'

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
# term_index is the column of the term and count_index is the column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

input_term = "becuase"
suggestions = sym_spell.lookup(input_term, Verbosity.CLOSEST, max_edit_distance=2, #Edit distance can be adjusted
                               transfer_casing=True, #Optional argument set to ignore case
                              include_unknown=True) #Return same word if unknown
for suggestion in suggestions:
    print(suggestion)  

because, 1, 271323986


In [159]:
# Creating function for finding unigram suggestions

def get_suggestions(word):
    if len(word) >= 4:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST,max_edit_distance=2, transfer_casing=True)
    else:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST,max_edit_distance=1, transfer_casing=True)
    return [str(x).split(',') for x in suggestions]

In [160]:
# Testing function

get_suggestions('becuase')

[['because', ' 1', ' 271323986']]

**Note**: The function has a variable edit distance: words of length 4 or more get edit distance of 2, shorter words get edit distance of 1. These preferences can be adjusted in the function if desired.

In [156]:
# Testing spelling suggestions with 'becuase of'
max_edit_distance_dictionary = 2
prefix_length = 7
sym_spell = SymSpell(max_edit_distance_dictionary, prefix_length)
bigram_path = pkg_resources.resource_filename("symspellpy", "frequency_bigramdictionary_en_243_342.txt")
if not sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1):
    print("Dictionary file not found")
if not sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2):
    print("Bigram dictionary file not found")
input_term = 'becuase of'
max_edit_distance_lookup = 2
suggestions = sym_spell.lookup_compound(input_term, max_edit_distance_lookup)
for suggestion in suggestions:
    print(suggestion) 

because of, 1, 3481714


In [157]:
# Creating function for finding bigram suggestions

def get_bigram_suggestions(bigram):
    suggestions = sym_spell.lookup_compound(bigram, max_edit_distance_lookup)
    for suggestion in suggestions:
        return [str(x).split(',') for x in suggestions] 

In [167]:
# Testing function
get_bigram_suggestions('worq harg')

[['work hard', ' 2', ' 53229']]

In [48]:
# Applying functions to create new column

misspell_df['suggestions'] =  misspell_df['misspelling'].apply(get_suggestions)
misspell_df.head()

,misspelling,lemma,POS,tok_lem_POS,freq,suggestions
0,alot,alot,NN,"(alot, alot, NN)",127,"[[lot, 1, 106405208], [slot, 1, 21602762],..."
1,studing,studing,VBG,"(studing, studing, VBG)",74,"[[studying, 1, 9763653], [studding, 1, 345..."
2,tofel,tofel,NNP,"(tofel, tofel, NNP)",66,"[[towel, 1, 4901668]]"
3,eilperin,eilperin,NNP,"(eilperin, eilperin, NNP)",48,[]
4,seollal,seollal,NNP,"(seollal, seollal, NNP)",47,[]


In [170]:
non_words.head(1)

,tok_lem_POS,misspelled_words,sentence
answer_id,,,
1,"[(i, i, PRP), (met, meet, VBD), (my, my, PRP$)...",[],[I met my friend Nife while I was studying in ...


In the above, the bigram function is first passed. If no suggestions are found, the unigram function is then passed. If there is still no suggestion, the original word is returned.

In [49]:
# Checking how many items without suggestions

len(misspell_df.loc[(misspell_df.suggestions.str.len() == 0),:])

1753

Items with no suggestions - these will be left in their original form though manual corrections could be applied if desired.

In [50]:
# Create new column with just the most likely correction (based on frequency)

misspell_df['correction'] = [x[0][0] if len(x) != 0 else np.NaN for x in misspell_df['suggestions']]

In [51]:
# If no correction, use original word

misspell_df.correction.fillna(misspell_df.misspelling, inplace=True)

In [52]:
misspell_df.head(1)

,misspelling,lemma,POS,tok_lem_POS,freq,suggestions,correction
0,alot,alot,NN,"(alot, alot, NN)",127,"[[lot, 1, 106405208], [slot, 1, 21602762],...",lot


In [53]:
# Create correction_POS column

misspell_df['correction_POS'] = list(zip(misspell_df.correction, misspell_df.POS))
misspell_df.head()

,misspelling,lemma,POS,tok_lem_POS,freq,suggestions,correction,correction_POS
0,alot,alot,NN,"(alot, alot, NN)",127,"[[lot, 1, 106405208], [slot, 1, 21602762],...",lot,"(lot, NN)"
1,studing,studing,VBG,"(studing, studing, VBG)",74,"[[studying, 1, 9763653], [studding, 1, 345...",studying,"(studying, VBG)"
2,tofel,tofel,NNP,"(tofel, tofel, NNP)",66,"[[towel, 1, 4901668]]",towel,"(towel, NNP)"
3,eilperin,eilperin,NNP,"(eilperin, eilperin, NNP)",48,[],eilperin,"(eilperin, NNP)"
4,seollal,seollal,NNP,"(seollal, seollal, NNP)",47,[],seollal,"(seollal, NNP)"


In [54]:
# As described earlier - one manual correction for 'alot' will be added

misspell_df.iloc[0,6] = 'a lot'
misspell_df.at[0, 'correction_POS'] = ('a','DT'),('lot','NN')

In [55]:
# Deleting unnecessary columns with duplicate information already contained in the tuple

del misspell_df['lemma']
del misspell_df['POS']

In [56]:
misspell_df.sort_values(by=['freq'], ascending=False).head(50)

,misspelling,tok_lem_POS,freq,suggestions,correction,correction_POS
0,alot,"(alot, alot, NN)",127,"[[lot, 1, 106405208], [slot, 1, 21602762],...",a lot,"((a, DT), (lot, NN))"
1,studing,"(studing, studing, VBG)",74,"[[studying, 1, 9763653], [studding, 1, 345...",studying,"(studying, VBG)"
2,tofel,"(tofel, tofel, NNP)",66,"[[towel, 1, 4901668]]",towel,"(towel, NNP)"
3,eilperin,"(eilperin, eilperin, NNP)",48,[],eilperin,"(eilperin, NNP)"
4,seollal,"(seollal, seollal, NNP)",47,[],seollal,"(seollal, NNP)"
5,goverment,"(goverment, goverment, NN)",47,"[[government, 1, 206582673]]",government,"(government, NN)"
6,decasia,"(decasia, decasia, NNP)",45,"[[decani, 2, 18542]]",decani,"(decani, NNP)"
7,iam,"(iam, iam, NNP)",43,"[[am, 1, 576436203], [aim, 1, 28951240], [...",am,"(am, NNP)"
8,misurata,"(misurata, misurata, NNP)",37,[],misurata,"(misurata, NNP)"
9,esi,"(esi, esi, NNP)",34,"[[est, 1, 58112143], [psi, 1, 6101411], [e...",est,"(est, NNP)"


### Incorporating corrections into `pelic_df`

In [57]:
# First removing items from misspell_df where no correction will take place

print(len(misspell_df))
misspell_df = misspell_df.loc[misspell_df.misspelling != misspell_df.correction]
print(len(misspell_df))

15779
13854


In [58]:
misspell_df.head()

,misspelling,tok_lem_POS,freq,suggestions,correction,correction_POS
0,alot,"(alot, alot, NN)",127,"[[lot, 1, 106405208], [slot, 1, 21602762],...",a lot,"((a, DT), (lot, NN))"
1,studing,"(studing, studing, VBG)",74,"[[studying, 1, 9763653], [studding, 1, 345...",studying,"(studying, VBG)"
2,tofel,"(tofel, tofel, NNP)",66,"[[towel, 1, 4901668]]",towel,"(towel, NNP)"
5,goverment,"(goverment, goverment, NN)",47,"[[government, 1, 206582673]]",government,"(government, NN)"
6,decasia,"(decasia, decasia, NNP)",45,"[[decani, 2, 18542]]",decani,"(decani, NNP)"


In [59]:
# Creating dictionary for mappying - key = incorrect spelling, value = correct spelling

misspell_dict = pd.Series(misspell_df.correction_POS.values,misspell_df.tok_lem_POS).to_dict()

In [60]:
misspell_dict

{('alot', 'alot', 'NN'): (('a', 'DT'), ('lot', 'NN')),
 ('studing', 'studing', 'VBG'): ('studying', 'VBG'),
 ('tofel', 'tofel', 'NNP'): ('towel', 'NNP'),
 ('goverment', 'goverment', 'NN'): ('government', 'NN'),
 ('decasia', 'decasia', 'NNP'): ('decani', 'NNP'),
 ('iam', 'iam', 'NNP'): ('am', 'NNP'),
 ('esi', 'esi', 'NNP'): ('est', 'NNP'),
 ('seol', 'seol', 'NNP'): ('sell', 'NNP'),
 ('nickell', 'nickell', 'NNP'): ('nickel', 'NNP'),
 ('thaksin', 'thaksin', 'NNP'): ('takin', 'NNP'),
 ('grammer', 'grammer', 'NN'): ('grammar', 'NN'),
 ('beatiful', 'beatiful', 'JJ'): ('beautiful', 'JJ'),
 ('okland', 'okland', 'NNP'): ('oakland', 'NNP'),
 ('becuase', 'becuase', 'NN'): ('because', 'NN'),
 ('finaly', 'finaly', 'NNP'): ('final', 'NNP'),
 ('oss', 'oss', 'NNP'): ('loss', 'NNP'),
 ('differents', 'differents', 'NNS'): ('different', 'NNS'),
 ('becouse', 'becouse', 'IN'): ('because', 'IN'),
 ('tgi', 'tgi', 'NNP'): ('tai', 'NNP'),
 ('resturant', 'resturant', 'NN'): ('restaurant', 'NN'),
 ('lifes', 'lif

In [61]:
# Incorporating back into pelic_df

pelic_df['tok_POS_corrected'] = pelic_df['tok_lem_POS'].apply\
(lambda row: [misspell_dict[(x[0].lower(),x[1],x[2])] if (x[0].lower(),x[1],x[2]) in misspell_dict else (x[0],x[2]) for x in row])

# One minor issue is that this will make misspelled items lower case when originally upper case.

In [62]:
# Checking with 'becuase'

print(pelic_df.loc[pelic_df.text.str.contains('becuase')].iloc[1,11]) #uncorrected
print(pelic_df.loc[pelic_df.text.str.contains('becuase')].iloc[1,12]) #corrected

(('My', 'my', 'PRP$'), ('friend', 'friend', 'NN'), ('is', 'be', 'VBZ'), ('realy', 'realy', 'JJ'), ('nise', 'nise', 'RB'), ('guy', 'guy', 'NN'), ('.', '.', '.'), ('I', 'i', 'PRP'), ('like', 'like', 'VBP'), ('hem', 'hem', 'JJ'), ('becuase', 'becuase', 'NN'), ('he', 'he', 'PRP'), ('is', 'be', 'VBZ'), ('friendlly', 'friendlly', 'RB'), ('and', 'and', 'CC'), ('lovliy', 'lovliy', 'NN'), ('.', '.', '.'))
[('My', 'PRP$'), ('friend', 'NN'), ('is', 'VBZ'), ('real', 'JJ'), ('nice', 'RB'), ('guy', 'NN'), ('.', '.'), ('I', 'PRP'), ('like', 'VBP'), ('hem', 'JJ'), ('because', 'NN'), ('he', 'PRP'), ('is', 'VBZ'), ('friendly', 'RB'), ('and', 'CC'), ('lovely', 'NN'), ('.', '.')]


We can see here that many approrpriate corrections have been made, including _beccuase_ -> _because_ , _nise_ -> _nice_ , and _lovily_ -> _lovely_ .  
Importantly, incorrect spellings that are actual words, e.g. _hem_ (should be _him_ in this case) are not corrected. In addition, as context is not considered, there will be some inaccuracies, e.g. _realy_ (marked as an adj) -> _real_ rather than _really_.

In [63]:
pelic_df.head()

,anon_id,L1,gender,course_id,level_id,class_id,question_id,version,text_len,text,tokens,tok_lem_POS,tok_POS_corrected
answer_id,,,,,,,,,,,,,
1,eq0,Arabic,Male,149,4,g,5,1,177,I met my friend Nife while I was studying in a...,"[I, met, my, friend, Nife, while, I, was, stud...","((I, i, PRP), (met, meet, VBD), (my, my, PRP$)...","[(I, PRP), (met, VBD), (my, PRP$), (friend, NN..."
2,am8,Thai,Female,149,4,g,5,1,137,"Ten years ago, I met a women on the train betw...","[Ten, years, ago, ,, I, met, a, women, on, the...","((Ten, ten, CD), (years, year, NNS), (ago, ago...","[(Ten, CD), (years, NNS), (ago, RB), (,, ,), (..."
3,dk5,Turkish,Female,115,4,w,12,1,64,In my country we usually don't use tea bags. F...,"[In, my, country, we, usually, do, n't, use, t...","((In, in, IN), (my, my, PRP$), (country, count...","[(In, IN), (my, PRP$), (country, NN), (we, PRP..."
4,dk5,Turkish,Female,115,4,w,13,1,6,I organized the instructions by time.,"[I, organized, the, instructions, by, time, .]","((I, i, PRP), (organized, organize, VBD), (the...","[(I, PRP), (organized, VBD), (the, DT), (instr..."
5,ad1,Korean,Female,115,4,w,12,1,59,"First, prepare a port, loose tea, and cup.\nSe...","[First, ,, prepare, a, port, ,, loose, tea, ,,...","((First, first, RB), (,, ,, ,), (prepare, prep...","[(First, RB), (,, ,), (prepare, VB), (a, DT), ..."


In [64]:
# Write out new PELIC_compiled.csv

pelic_df.to_csv('PELIC_compiled_spellcorrected.csv', encoding='utf-8', index=True)

In [65]:
# Pickle new pelic_df dataframe

pelic_df.to_pickle('pelic_spellcorrected.pkl')

If preferred, this entire spelling correctin process can also be applied to [`answer.csv`]() instead of `PELIC_compiled`.

[Back to top](#Corrected-spelling)